text-embedding-ada-002	Demensions: 1536	Max Tokens: 8191

text-embedding-3-small	Demensions: 1536	Max Tokens: 8191

text-embedding-3-large	Demensions: 3072	Max Tokens: 8191

# Langchain 활용

In [39]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import CharacterTextSplitter

from pinecone import Pinecone
from dotenv import load_dotenv
import os

# 환경변수 불러와
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

# 파인콘 정의해
index_name= "ragcontract"
embeddings = OpenAIEmbeddings()

embeddings = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=3072)

In [3]:
loader = TextLoader("./source/state_of_the_union.txt")
documents = loader.load()
# documents = [Document(metadata={'source': './source/state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. ..... 
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [56]:
vectorstore_from_docs = PineconeVectorStore.from_documents(
    docs,
    index_name=index_name,
    embedding=embeddings,
    namespace="example"
)

In [57]:
query = "Who is Ketanji Brown Jackson?"
vectorstore_from_docs.similarity_search(query)


[Document(metadata={'source': './source/state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(metadata={'source': './source/state_of_the_union.txt'}, page_content='A former top litigator in private practice.

In [ ]:
namespace = "example"
results = vectorstore_from_docs.similarity_search(query,k=5, namespace = namespace)
print(len(results))
for result in results:

    print(result)

In [47]:
from pinecone import Pinecone

# Pinecone API로 쓰자면
pc = Pinecone()
index = pc.Index("ragcontract")

In [51]:
index.query(
    namespace="Default",
    vector=[0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
    top_k=2,
    include_values=True,
    include_metadata=True,
    # filter={"genre": {"$eq": "action"}}
)

{'matches': [], 'namespace': 'Default', 'usage': {'read_units': 1}}

In [53]:
print(index.describe_index_stats())

{'dimension': 3072,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 45}},
 'total_vector_count': 45}


In [54]:
query_results1 = index.query(
    namespace="ns1",
    vector=[1.0, 1.5],
    top_k=3,
    include_values=True
)

print(query_results1)


{'matches': [], 'namespace': 'ns1', 'usage': {'read_units': 1}}
